### Io type

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split        #split data for training and testing
from sklearn.metrics import accuracy_score                  #to view accuracy
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow.python.ops import array_ops, math_ops      #for math operations division_no_nan
from tensorflow.keras.layers import *
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from sklearn.metrics import roc_curve, auc
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

from qkeras import *
import hls4ml
from hls4ml.model.profiling import numerical, activations_keras, boxplot

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10,10]
plt.rcParams['font.size'] = 16.0

seed = 48

np.random.seed(seed)
tf.random.set_seed(seed)

import os
import sys

# will need to clone https://github.com/kpidgeon/cms-l1-triggers for a few
# helper functions if running notebook and include path to repo here
sys.path.append('/usersc/bz18310/previous_notebook/cms-l1-triggers')

from utils.analysis import eff_rate, optimal_eff_rate
from utils.preprocessing import resize
from utils.plotting import *
from utils.hls4ml_helpers import *

plt.rc('figure', figsize=(8,6))


###for loading and saving models
from tensorflow.keras.models import load_model

from qkeras.utils import model_save_quantized_weights
from qkeras.utils import load_qmodel

###For Pruning summaries

import tempfile

In [2]:
model = tf.keras.models.load_model('./cnn_20x12_keras_trained')   #load trained model

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 18, 10, 4)         36        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 5, 4)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 5, 4)           16        
_________________________________________________________________
relu_c1 (Activation)         (None, 9, 5, 4)           0         
_________________________________________________________________
conv2 (Conv2D)               (None, 7, 3, 8)           288       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 1, 8)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 1, 8)           3

In [9]:
def hls4ml_converter(model_name, directory, io_type, ReuseFactor):
    
    config = hls4ml.utils.config_from_keras_model(model_name, granularity='model')
    print("-----------------------------------")
    config = {'Model': {'Precision': 'ap_fixed<5,1>',
              'ReuseFactor': ReuseFactor,
              'Strategy': 'Latency'}}
    print("Configuration")
    display(config)
    print("-----------------------------------")
    hls_model = hls4ml.converters.convert_from_keras_model(model_name,
                                                           hls_config=config,   #set configuration of model
                                                           output_dir=directory,  #6bit model save
                                                           fpga_part='xcku15p-ffva1760-2-e',     #What?
                                                            clock_period=(1/.24),      #set clock period??
                                                              io_type=io_type)     #
    hls_model.compile()
    return hls_model


In [23]:
#hls_io_stream = hls4ml_converter(model, './hls_parameters/io_stream/hls4ml_prj', io_type='io_stream')
hls_io_parallel = hls4ml_converter(model, './hls_parameters/io_parallel/hls4ml_prj', io_type='io_parallel')
#hls_io_serial = hls4ml_converter(model, './hls_parameters/io_serial/hls4ml_prj', io_type='io_serial')

Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: Input
Layer name: conv1, layer type: Conv2D
  -> Activation (linear), layer name: conv1
Layer name: max_pooling2d, layer type: MaxPooling2D
Layer name: batch_normalization, layer type: BatchNormalization
Layer name: relu_c1, layer type: Activation
Layer name: conv2, layer type: Conv2D
  -> Activation (linear), layer name: conv2
Layer name: max_pooling2d_1, layer type: MaxPooling2D
Layer name: batch_normalization_1, layer type: BatchNormalization
Layer name: relu_c2, layer type: Activation
Layer name: dense1, layer type: Dense
  -> Activation (linear), layer name: dense1
Layer name: batch_normalization_2, layer type: BatchNormalization
Layer name: relu1, layer type: Activation
Layer name: output, layer type: Dense
  -> Activation (linear), layer name: output
Layer name: batch_normalization_3, layer type: BatchNormalization
Layer name: sigmoid, layer type: Activation
-----------------------------------
Configuration


{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: InputLayer, current shape: [[None, 20, 12, 1]]
Layer name: conv1, layer type: Conv2D, current shape: [[None, 20, 12, 1]]
Layer name: max_pooling2d, layer type: MaxPooling2D, current shape: [[None, 18, 10, 4]]
Layer name: batch_normalization, layer type: BatchNormalization, current shape: [[None, 9, 5, 4]]
Layer name: relu_c1, layer type: Activation, current shape: [[None, 9, 5, 4]]
Layer name: conv2, layer type: Conv2D, current shape: [[None, 9, 5, 4]]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, current shape: [[None, 7, 3, 8]]
Layer name: batch_normalization_1, layer type: BatchNormalization, current shape: [[None, 3, 1, 8]]
Layer name: relu_c2, layer type: Activation, current shape: [[None, 3, 1, 8]]
Layer name: dense1, layer type: Dense, current shape: [[None, 3, 1, 8]]
Layer name: batch_normalization_2, layer type: BatchNormalization, current shape: [[None, 24]]
Lay

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: InputLayer, current shape: [[None, 20, 12, 1]]
Layer name: conv1, layer type: Conv2D, current shape: [[None, 20, 12, 1]]
Layer name: max_pooling2d, layer type: MaxPooling2D, current shape: [[None, 18, 10, 4]]
Layer name: batch_normalization, layer type: BatchNormalization, current shape: [[None, 9, 5, 4]]
Layer name: relu_c1, layer type: Activation, current shape: [[None, 9, 5, 4]]
Layer name: conv2, layer type: Conv2D, current shape: [[None, 9, 5, 4]]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, current shape: [[None, 7, 3, 8]]
Layer name: batch_normalization_1, layer type: BatchNormalization, current shape: [[None, 3, 1, 8]]
Layer name: relu_c2, layer type: Activation, current shape: [[None, 3, 1, 8]]
Layer name: dense1, layer type: Dense, current shape: [[None, 3, 1, 8]]
Layer name: batch_normalization_2, layer type: BatchNormalization, current shape: [[None, 24]]
Lay

In [24]:
#hls_io_stream.build(csim=False)
#hls_io_parallel.build(csim=False)
#hls_io_serial.build(csim=False)

Synthesis report not found.


{}

In [15]:
hls4ml.report.read_vivado_report('./hls_parameters/io_parallel/hls4ml_prj')

Found 1 solution(s) in ./hls_parameters/io_parallel/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
Synthesis report not found.
Co-simulation report not found.


In [12]:
ReuseFactors = [256, 512, 1024, 2048, 4096]

a = './hls_parameters/io_parallel'

directories = []

def remove_spaces(string):
    a = string.replace(",", "_")
    return a.replace(" ", "")

for r in ReuseFactors:
    b = '_r'
    b += str(r) 
    directories.append(a + b + '/hls4ml_prj')  

In [13]:
for i,r in enumerate(ReuseFactors):
    hls_io_parallel = hls4ml_converter(model, directories[i], io_type='io_parallel', ReuseFactor=r)
    hls_io_parallel.build(csim=False)

Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: Input
Layer name: conv1, layer type: Conv2D
  -> Activation (linear), layer name: conv1
Layer name: max_pooling2d, layer type: MaxPooling2D
Layer name: batch_normalization, layer type: BatchNormalization
Layer name: relu_c1, layer type: Activation
Layer name: conv2, layer type: Conv2D
  -> Activation (linear), layer name: conv2
Layer name: max_pooling2d_1, layer type: MaxPooling2D
Layer name: batch_normalization_1, layer type: BatchNormalization
Layer name: relu_c2, layer type: Activation
Layer name: dense1, layer type: Dense
  -> Activation (linear), layer name: dense1
Layer name: batch_normalization_2, layer type: BatchNormalization
Layer name: relu1, layer type: Activation
Layer name: output, layer type: Dense
  -> Activation (linear), layer name: output
Layer name: batch_normalization_3, layer type: BatchNormalization
Layer name: sigmoid, layer type: Activation
-----------------------------------
Configuration


{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 256,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: InputLayer, current shape: [[None, 20, 12, 1]]
Layer name: conv1, layer type: Conv2D, current shape: [[None, 20, 12, 1]]
Layer name: max_pooling2d, layer type: MaxPooling2D, current shape: [[None, 18, 10, 4]]
Layer name: batch_normalization, layer type: BatchNormalization, current shape: [[None, 9, 5, 4]]
Layer name: relu_c1, layer type: Activation, current shape: [[None, 9, 5, 4]]
Layer name: conv2, layer type: Conv2D, current shape: [[None, 9, 5, 4]]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, current shape: [[None, 7, 3, 8]]
Layer name: batch_normalization_1, layer type: BatchNormalization, current shape: [[None, 3, 1, 8]]
Layer name: relu_c2, layer type: Activation, current shape: [[None, 3, 1, 8]]
Layer name: dense1, layer type: Dense, current shape: [[None, 3, 1, 8]]
Layer name: batch_normalization_2, layer type: BatchNormalization, current shape: [[None, 24]]
Lay

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 512,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: InputLayer, current shape: [[None, 20, 12, 1]]
Layer name: conv1, layer type: Conv2D, current shape: [[None, 20, 12, 1]]
Layer name: max_pooling2d, layer type: MaxPooling2D, current shape: [[None, 18, 10, 4]]
Layer name: batch_normalization, layer type: BatchNormalization, current shape: [[None, 9, 5, 4]]
Layer name: relu_c1, layer type: Activation, current shape: [[None, 9, 5, 4]]
Layer name: conv2, layer type: Conv2D, current shape: [[None, 9, 5, 4]]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, current shape: [[None, 7, 3, 8]]
Layer name: batch_normalization_1, layer type: BatchNormalization, current shape: [[None, 3, 1, 8]]
Layer name: relu_c2, layer type: Activation, current shape: [[None, 3, 1, 8]]
Layer name: dense1, layer type: Dense, current shape: [[None, 3, 1, 8]]
Layer name: batch_normalization_2, layer type: BatchNormalization, current shape: [[None, 24]]
Lay

...

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: InputLayer, current shape: [[None, 20, 12, 1]]
Layer name: conv1, layer type: Conv2D, current shape: [[None, 20, 12, 1]]
Layer name: max_pooling2d, layer type: MaxPooling2D, current shape: [[None, 18, 10, 4]]
Layer name: batch_normalization, layer type: BatchNormalization, current shape: [[None, 9, 5, 4]]
Layer name: relu_c1, layer type: Activation, current shape: [[None, 9, 5, 4]]
Layer name: conv2, layer type: Conv2D, current shape: [[None, 9, 5, 4]]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, current shape: [[None, 7, 3, 8]]
Layer name: batch_normalization_1, layer type: BatchNormalization, current shape: [[None, 3, 1, 8]]
Layer name: relu_c2, layer type: Activation, current shape: [[None, 3, 1, 8]]
Layer name: dense1, layer type: Dense, current shape: [[None, 3, 1, 8]]
Layer name: batch_normalization_2, layer type: BatchNormalization, current shape: [[None, 24]]
Lay

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 2048,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: InputLayer, current shape: [[None, 20, 12, 1]]
Layer name: conv1, layer type: Conv2D, current shape: [[None, 20, 12, 1]]
Layer name: max_pooling2d, layer type: MaxPooling2D, current shape: [[None, 18, 10, 4]]
Layer name: batch_normalization, layer type: BatchNormalization, current shape: [[None, 9, 5, 4]]
Layer name: relu_c1, layer type: Activation, current shape: [[None, 9, 5, 4]]
Layer name: conv2, layer type: Conv2D, current shape: [[None, 9, 5, 4]]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, current shape: [[None, 7, 3, 8]]
Layer name: batch_normalization_1, layer type: BatchNormalization, current shape: [[None, 3, 1, 8]]
Layer name: relu_c2, layer type: Activation, current shape: [[None, 3, 1, 8]]
Layer name: dense1, layer type: Dense, current shape: [[None, 3, 1, 8]]
Layer name: batch_normalization_2, layer type: BatchNormalization, current shape: [[None, 24]]
Lay

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 4096,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: InputLayer, current shape: [[None, 20, 12, 1]]
Layer name: conv1, layer type: Conv2D, current shape: [[None, 20, 12, 1]]
Layer name: max_pooling2d, layer type: MaxPooling2D, current shape: [[None, 18, 10, 4]]
Layer name: batch_normalization, layer type: BatchNormalization, current shape: [[None, 9, 5, 4]]
Layer name: relu_c1, layer type: Activation, current shape: [[None, 9, 5, 4]]
Layer name: conv2, layer type: Conv2D, current shape: [[None, 9, 5, 4]]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, current shape: [[None, 7, 3, 8]]
Layer name: batch_normalization_1, layer type: BatchNormalization, current shape: [[None, 3, 1, 8]]
Layer name: relu_c2, layer type: Activation, current shape: [[None, 3, 1, 8]]
Layer name: dense1, layer type: Dense, current shape: [[None, 3, 1, 8]]
Layer name: batch_normalization_2, layer type: BatchNormalization, current shape: [[None, 24]]
Lay

In [15]:
for d in directories:
    hls4ml.report.read_vivado_report(d)

Found 1 solution(s) in ./hls_parameters/io_parallel_r256/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
Synthesis report not found.
Co-simulation report not found.
Found 1 solution(s) in ./hls_parameters/io_parallel_r512/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
Synthesis report not found.
Co-simulation report not found.
Found 1 solution(s) in ./hls_parameters/io_parallel_r1024/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
Synthesis report not found.
Co-simulation report not found.
Found 1 solution(s) in ./hls_parameters/io_parallel_r2048/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
Synthesis report not found.
Co-simulation report not found.
Found 1 solution(s) in ./hls_parameters/io_parallel_r4096/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
Synthesis re